In [23]:
import pandas as pd

# read in the main csv file
df = pd.read_csv("../all_csv_images/all_data_all_images_no_duplicates.csv")

# select 100 random rows from the dataframe
test_df = df.sample(n=50)

# write the selected rows to a new csv file
test_df.to_csv('../all_csv_images/test_file.csv', index=False)


In [24]:
import pandas as pd
import pytesseract
from PIL import Image
from tqdm import tqdm
import logging
from PIL import Image, ImageEnhance

# Set the path to the Tesseract-OCR executable
pytesseract.pytesseract.tesseract_cmd = (
    r"C:\Program Files\Tesseract-OCR\tesseract.exe"
)

# Configure logging
logging.basicConfig(filename='ocr.log', level=logging.DEBUG)

# Read in the CSV file containing the file names
df = pd.read_csv('../all_csv_images/test_file.csv')

# Add a new column to the DataFrame to store the OCR text
df['ocr_text'] = ''

with open("ocr_log.txt", "w") as log:
    # Iterate over each file name in the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
           
# Open the image using PIL
            image = Image.open(row['relative_path'])

# Convert the image to grayscale
            image = image.convert("L")
            # Apply contrast stretching to the image
            contrast = ImageEnhance.Contrast(image)
            image = contrast.enhance(2)

# Use PyTesseract to perform OCR on the image
            ocr_text = pytesseract.image_to_string(image)
            # Store the OCR text in the corresponding row of the new column
            df.at[index, 'ocr_text'] = ocr_text
        except Exception as e:
            # Log any errors that occur during the OCR process
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')

# Write the updated DataFrame to a new CSV file
df.to_csv('../all_csv_images/ocr_PIL_enhance.csv', index=False)


Performing OCR: 100%|██████████| 50/50 [00:25<00:00,  1.97it/s]


In [25]:
df["ocr_text"][1]

"When you're trynna get lobotomized i in\npeace but drunk-ass Gary is ranting\nabout his wife... like shit Gary just get a\ndivorce if you care that much, I'm trynna\nget my lobotomy game on and you\nStraight souring the vibe right now\n"

In [26]:
import pandas as pd
import pytesseract
from PIL import Image
from tqdm import tqdm
import logging
import cv2

# Set the path to the Tesseract-OCR executable
pytesseract.pytesseract.tesseract_cmd = (
    r"C:\Program Files\Tesseract-OCR\tesseract.exe"
)

# Configure logging
logging.basicConfig(filename='ocr.log', level=logging.DEBUG)

# Read in the CSV file containing the file names
df = pd.read_csv('../all_csv_images/test_file.csv')

# Add a new column to the DataFrame to store the OCR text
df['ocr_text'] = ''

with open("ocr_log.txt", "w") as log:
    # Iterate over each file name in the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
            # Open the image using OpenCV
            image = cv2.imread(row['relative_path'])
            # preprocess the image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 3)
            gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            # Use PyTesseract to perform OCR on the image
            ocr_text = pytesseract.image_to_string(gray)
            # Store the OCR text in the corresponding row of the new column
            df.at[index, 'ocr_text'] = ocr_text
        except Exception as e:
            # Log any errors that occur during the OCR process
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')

# Write the updated DataFrame to a new CSV file
df.to_csv('../all_csv_images/ocr_cv2.csv', index=False)

Performing OCR: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


In [27]:
df["ocr_text"][1]

"When you're trynna get lobotomized in\npeace but drunk-ass Gary is ranting\nabout his wife... like shit Gary just get a\ndivorce if you care that much, I'm trynna\nget my lobotomy game on and you\nStraight souring the vibe right now\n\n"

In [29]:
import pandas as pd
import io
from google.cloud import vision
from google.oauth2 import service_account
from tqdm import tqdm
import logging

# Configure logging
logging.basicConfig(filename='ocr.log', level=logging.DEBUG)

# Read in the CSV file containing the file names
df = pd.read_csv('../all_csv_images/all_data_all_images_no_duplicates.csv')

# Add a new column to the DataFrame to store the OCR text
df['ocr_text'] = ''
# credentials = "C:/Users/Alexa/OneDrive/Desktop/Forschungsseminar/Reddit_Memes/Code/core-gearbox-375812-faa4c37a689b.json"
credentials = service_account.Credentials.from_service_account_file('C:/Users/Alexa/OneDrive/Desktop/Forschungsseminar/Reddit_Memes/Code/core-gearbox-375812-faa4c37a689b.json')
# Authenticate with the Google Cloud Vision API
client = vision.ImageAnnotatorClient(credentials=credentials)

with open("ocr_log.txt", "w") as log:
    # Iterate over each file name in the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
            # Read image into memory
            with io.open(row['relative_path'], 'rb') as image_file:
                content = image_file.read()

            # Perform OCR using the Google Vision API
            image = vision.Image(content=content)
            response = client.text_detection(image=image)
            ocr_text = response.text_annotations
            # Store the OCR text in the corresponding row of the new column
            df.at[index, 'ocr_text'] = ocr_text[0].description
        except Exception as e:
            # Log any errors that occur during the OCR process
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')

    # Write the updated DataFrame to a new CSV file
    df.to_csv('../all_csv_images/ocr_google_vision.csv', index=False)


Performing OCR: 100%|██████████| 28052/28052 [6:08:50<00:00,  1.27it/s]   
